In [50]:
#Uses python3

import numpy as np
import pandas as pd 
import time
import matplotlib
import matplotlib.pyplot as plt
import sys

In [62]:
#====================== all functions ======================
class BW:
    def __init__(self, L, state):
        self.L = L
        self.J = 1    
        self.N_Relax = 90
        self.N_mc = 1
        self.N_avg =  10
        self.state = state 
        self.mask = create_mask(L)
        self.edict = gen_e_dict()
        
        self.e = []    
        self.m = []  
        
    def clear(self):
        """Reinitialize the data structures for the next query after the previous query."""
        self.e = []    
        self.m = [] 
        pass
    
    def get_conf(self,i,j):
        """формируем конфигурацию, по вышеописанному правилу таблицы энергий"""
        s = self.state
        arr = ([s[i,j]]+[s[i-1,j]]+[s[i,j+1]]+[s[i+1,j+1]]+
               [s[i+1,j]]+[s[i,j-1]]+[s[i-1,j-1]])
        return arr
    
    def flip(self,j,k):
        """перевернуть спин j,k"""
        self.state[j,k] *= -1
        pass
        
    def step(self, K):
        """perform 1 Metropolis step"""
        j,k = np.random.randint(-1,self.L-1), np.random.randint(-1,self.L-1)
        arr = self.get_conf(j,k)
        e_jk = self.edict[str(arr)]
        if e_jk > 0:
            self.flip(j,k)
        elif np.random.uniform() <= np.exp(2*e_jk*K): #uniformly distributed over interval [low, high)
            self.flip(j,k)
        pass
    
    def mc_step(self, K):
        """perform L*L flips for 1 MC step"""
        for _ in range(self.L**2):
            self.step(K)
        pass
            
    def calc_e(self, st):
        """calculate energy per site
            # expland state matrix"""
        a = np.concatenate((st[self.L-1].reshape(1,self.L), st, st[0].reshape(1,self.L)), axis=0) 
        b = np.concatenate((a[:,-1].reshape(self.L+2,1),a,a[:,0].reshape(self.L+2,1)), axis=1)
        return -np.sum(b[1:-1, 1:-1]*b[2:, 2:]*(b[2:, 1:-1]+b[1:-1, 2:]))/(self.L*self.L)  

    def calc_ms(self, st):
        """magnetization"""
        msr = np.array([np.sum(st[self.mask==i]) for i in [0,1,2]])/(self.L*self.L)
        return np.sqrt(np.sum(msr*msr))
    
    def model_em(self,T):
        self.clear()
        K = self.J/T
        #relax be4 AVG
        for x in range(self.N_Relax):
            self.mc_step(K)
        for _ in range(self.N_avg):        
            for __ in range(self.N_mc):  
                self.mc_step(K)
            self.e += [self.calc_e(self.state)]
            self.m += [self.calc_ms(self.state)]
        return self.e, self.m
        
    
def gen_state(L):
    """generate random init. state with lenght L*L and q=[-1,1]"""
    np.random.seed(1)
    return np.array([np.random.choice([-1,1]) for _ in range(L*L)]).reshape(L,L)

def create_mask(L):
    """маска в виде 3 под-решёток"""
    a = np.asarray([i % 3 for i in range(L)])
    return (a + a[:, None])%3

def gen_e0(n):
    """генерирует все возможные последовательности из 7 спинов"""
    n_b = str(bin(n))[2:]
    n_arr = [-1]*(7-len(n_b)) + [int(x) if x=='1' else -1 for x in n_b]
    return n_arr

def calc_e_dict(arr):
    """вычисляет энергию конкретной конфигурации"""
    summa = (
        arr[1]*arr[2] + arr[2]*arr[3] + arr[3]*arr[4] +
        arr[4]*arr[5] + arr[5]*arr[6] + arr[6]*arr[1]
    )
    return -arr[0]*summa

def gen_e_dict():
    """генерируем и заоплняем словарь всех энергий"""
    d = dict()
    for i in range(0,128):
        conf = gen_e0(i)
        d[str(conf)] = calc_e_dict(conf)
    return d


In [ ]:
if __name__ == '__main__':
    L = 81
    rnd.seed(int(sys.argv[1]))
    np.random.seed(int(sys.argv[1]))
    df_e, df_m = [pd.DataFrame() for i in range(2)]
    bw = BW(L, gen_state(L))

    tc = 1/(np.log(2**0.5+1)/2) # 2.269185314213022
    t_ = np.array([0.001, 0.002, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3])
    t = np.round(-t_*tc+tc, 3)  #low
    # t_high = np.round(t_*tc+tc, 3)
    # t = np.concatenate((t_low, t_high), axis=None)
    t.sort()

    st = time.time()
    for ind,T in enumerate(t):
        e,m = bw.model_em(T)
        df_e.insert(ind,T,e, True)
        df_m.insert(ind,T,m, True)
 
    title = 'bw_L'+str(L)+'_avg'+str(bw.N_avg)+'_mc'+str(bw.N_mc)+'_after'+str(bw.N_Relax)+'mc_'
    df_e.to_csv('export/e_'+title+'seed'+str(sys.argv[1])+'.csv', index = None, header=True)
    df_m.to_csv('export/m_'+title+'seed'+str(sys.argv[1])+'.csv', index = None, header=True)
    print('im done in ',time.time()-st)

In [60]:
global L, q, J
L = 100
q = 4      # components
J = 1      # interaction energy
bw = BW(L, gen_state(L))

%timeit [bw.mc_step(1) for _ in range(10)]

2.16 s ± 52.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
global L, q, J
L = 100
J = 1      # interaction energy
bw = BW(L, gen_state(L))
print(bw.calc_e(bw.state))

for _ in range(10):
    bw.mc_step(1)
print(bw.calc_e(bw.state))

-0.0048
-1.1


In [61]:
# py     -- 2.16 s ± 52.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)     # original

v2